In [1]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import accuracy_score

import numpy as np
import matplotlib.pyplot as plt

from cv_svm import SVM_smooth

#import matplotlib
#matplotlib.use("pgf")
#matplotlib.rcParams.update({
#    "pgf.texsystem": "pdflatex",
#    'font.family': 'serif',
#    'text.usetex': True,
#    'pgf.rcfonts': False,
#})

In [2]:
#X, y = load_breast_cancer(return_X_y=True)
X, y = make_classification(n_samples=250, n_features=50)
n = X.shape[0]
p = X.shape[1]
y[np.where(y == 0)] = -1

In [3]:
X = StandardScaler().fit_transform(X)
print(X.shape)

(250, 50)


In [4]:
clf = SVC(kernel='linear')
clf.fit(X, y)
sk_coef = clf.coef_/np.linalg.norm(clf.coef_)
print(clf.coef_)
print(clf.intercept_)

[[-0.03580252  0.0538849  -0.41413796  0.13782758 -0.08045537  0.25578288
   0.16441577  0.07234894  0.03997236  0.07649082  0.13801625  0.04621733
   0.15617996 -0.07805715  0.0880955   0.06432917 -0.18635694  0.31409921
   0.11841862  0.12196187  0.28957633  0.93585554  0.26729545  0.23658273
  -0.14781746 -0.03936862  0.17181509  0.43683484  0.28172153 -0.3324809
  -0.49175585  0.04863838  0.06841735 -0.88860019 -0.06806724 -0.20197403
  -0.08915705 -0.45079492  0.29000417  0.0806888  -0.05866046  0.09644375
  -0.34502548  0.13634357 -0.14710968  1.13248734  0.26767526  0.00951547
   0.040759   -0.15436103]]
[0.05971027]


In [5]:
y_pred = clf.predict(X)
accuracy_score(y, y_pred)

0.936

In [6]:
clf = SVM_smooth(sigma=2e-1, lbd=0)
clf.fit(X, y, thresh=1e-3, n_iter=2500, eta=0.95/n, approx_cv=True, cv=True, log_iter=True, log_iacv=True)
print(f"grad {np.linalg.norm(clf.nabla_fgd_(X, y, clf.weights_, clf.sigma_, clf.lbd_))}")
coef = clf.weights_/np.linalg.norm(clf.weights_)
print(f"diff {np.mean(np.abs(coef - sk_coef))} | pct {np.mean(np.abs(coef - sk_coef))/np.sum(np.abs(coef))}")

y_pred = clf.predict(X)
accuracy_score(y, y_pred)

iter 0 | grad 1.32188 IACV: 0.00000000 | baseline: 0.00500413
iter 1 | grad 1.32159 IACV: 0.00000001 | baseline: 0.00498614
iter 2 | grad 1.32129 IACV: 0.00000003 | baseline: 0.00497029
iter 3 | grad 1.32098 IACV: 0.00000007 | baseline: 0.00495659
iter 4 | grad 1.32066 IACV: 0.00000011 | baseline: 0.00494505
iter 5 | grad 1.32032 IACV: 0.00000017 | baseline: 0.00493570
iter 6 | grad 1.31998 IACV: 0.00000024 | baseline: 0.00492855
iter 7 | grad 1.31962 IACV: 0.00000033 | baseline: 0.00492359
iter 8 | grad 1.31925 IACV: 0.00000043 | baseline: 0.00492084
iter 9 | grad 1.31886 IACV: 0.00000054 | baseline: 0.00492030
iter 10 | grad 1.31846 IACV: 0.00000067 | baseline: 0.00492197
iter 11 | grad 1.31804 IACV: 0.00000081 | baseline: 0.00492583
iter 12 | grad 1.31760 IACV: 0.00000097 | baseline: 0.00493189
iter 13 | grad 1.31714 IACV: 0.00000115 | baseline: 0.00494014
iter 14 | grad 1.31667 IACV: 0.00000135 | baseline: 0.00495055
iter 15 | grad 1.31617 IACV: 0.00000157 | baseline: 0.00496312
it

KeyboardInterrupt: 

In [ ]:
iacv_coef = normalize(clf.loo_iacv_, axis=1)
true_coef = normalize(clf.loo_true_, axis=1)
print(np.mean(np.linalg.norm(iacv_coef - true_coef, 2, axis=1)))
print(np.mean(np.linalg.norm(coef - true_coef, 2, axis=1)))

In [ ]:
print(iacv_coef)
print(true_coef)

In [ ]:
print(np.std(clf.loo_iacv_))
print(np.std(clf.loo_true_))

In [ ]:
print(np.mean(clf.loo_iacv_, axis=0))
print(np.mean(clf.loo_true_, axis=0))

In [ ]:
plt.hist(np.mean(clf.loo_iacv_, axis=0), alpha=0.5, bins=30, label="IACV")
plt.hist(np.mean(clf.loo_true_, axis=0), alpha=0.5, bins=30, label="true")
plt.legend()

In [ ]:
clf.loo_iacv_

In [ ]:
clf.loo_true_

In [ ]:
iacv_mean = np.mean(clf.loo_iacv_, axis=0)
coef = iacv_mean/np.linalg.norm(iacv_mean)
print(f"diff {np.mean(np.abs(coef - sk_coef))} | pct {np.mean(np.abs(coef - sk_coef))/np.sum(np.abs(coef))}")

In [ ]:
true_cv_mean = np.mean(clf.loo_true_, axis=0)
coef = true_cv_mean/np.linalg.norm(true_cv_mean)
print(f"diff {np.mean(np.abs(coef - sk_coef))} | pct {np.mean(np.abs(coef - sk_coef))/np.sum(np.abs(coef))}")

In [ ]:
coef = clf.weights_/np.linalg.norm(clf.weights_)
nbin = 100
plt.hist(coef, bins=nbin, label="FGD", alpha=0.3)
plt.hist(sk_coef.flatten(), bins=nbin, label="sklearn", alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# test different sigma values
import time
#sigmas = [2e-30, 2e-25, 2e-15, 2e-10, 2e-5, 2e-1]
sigmas = [2e-15, 2e-10, 2e-5, 2e-3, 2e-1, 5e-1, 7e-1, 8e-1, 1, 1.25, 1.5, 2, 2.5, 5]
scores = []

start = time.time()
for s in sigmas:
    if s == 0:
        continue
    clf = SVM_smooth(sigma=s, lbd=1e-35)
    #clf.fit(X, y, thresh=5e-3, n_iter=1000, eta=0.5 * s, cv=False, approx_cv=True, log_iacv=False, log_iter=True)
    print(f"running experiment for sigma = {s}")
    clf.fit(X, y, thresh=1e-3, n_iter=3500, eta=0.85/n, approx_cv=True, cv=True, log_iacv=False, log_iter=False)
    y_pred = clf.predict(X)
    score = accuracy_score(y, y_pred)

    coef = clf.weights_/np.linalg.norm(clf.weights_)

    #print(clf.weights_)
    print(f"\tsigma {s} | score {score} | grad {np.linalg.norm(clf.nabla_fgd_(X, y, clf.weights_, clf.sigma_, clf.lbd_))} | sklearn diff {np.mean(np.abs(coef- sk_coef))}")
    print(f"\tIACV: {np.mean(np.linalg.norm(clf.loo_iacv_ - clf.loo_true_, 2, axis=1))} | baseline: {np.mean(np.linalg.norm(clf.weights_ - clf.loo_true_, 2, axis=1))}" )
    scores.append([np.mean(np.linalg.norm(clf.loo_iacv_ - clf.loo_true_, 2, axis=1)), np.mean(np.linalg.norm(clf.weights_ - clf.loo_true_, 2, axis=1))])

end = time.time()
print(end - start)

In [ ]:
print(scores)
scores = np.asarray(scores)

In [ ]:
fig, ax = plt.subplots()
ax.plot(sigmas, scores[:, 0], label="IACV", c='black')
ax.plot(sigmas, scores[:, 1], label="baseline", c='green', linestyle="--")
ax.set_xscale('log')
ax.set_xlabel("$\log(\sigma)$")
ax.set_ylabel("Err Approx")
ax.legend()
ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
#plt.savefig('svmtest_sigma_err_approx_more_iter.pgf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(sigmas, scores[:, 0] - scores[:, 1])
plt.show()

In [9]:
# test different lambda values
import time
lambdas = np.linspace(0, 1.5, 10)
lbd_scores = []

start = time.time()
for l in lambdas:
    clf = SVM_smooth(sigma=2e-5, lbd=l)
    #clf.fit(X, y, thresh=5e-3, n_iter=1000, eta=0.5 * s, cv=False, approx_cv=True, log_iacv=False, log_iter=True)
    print(f"running experiment for lambda = {l}")
    clf.fit(X, y, thresh=1e-3, n_iter=3500, eta=0.5/n, approx_cv=True, cv=True, log_iacv=False, log_iter=False)
    y_pred = clf.predict(X)
    score = accuracy_score(y, y_pred)

    coef = clf.weights_/np.linalg.norm(clf.weights_)

    #print(clf.weights_)
    print(f"\tlambda {l} | score {score} | grad {np.linalg.norm(clf.nabla_fgd_(X, y, clf.weights_, clf.sigma_, clf.lbd_))} | sklearn diff {np.mean(np.abs(coef- sk_coef))}")
    print(f"\tIACV: {np.mean(np.linalg.norm(clf.loo_iacv_ - clf.loo_true_, 2, axis=1))} | baseline: {np.mean(np.linalg.norm(clf.weights_ - clf.loo_true_, 2, axis=1))}" )
    lbd_scores.append([np.mean(np.linalg.norm(clf.loo_iacv_ - clf.loo_true_, 2, axis=1)), np.mean(np.linalg.norm(clf.weights_ - clf.loo_true_, 2, axis=1))])

end = time.time()
print(end - start)

running experiment for lambda = 0.0
	lambda 0.0 | score 0.92 | grad 0.08921319718170927 | sklearn diff 0.04628760779484646
	IACV: 5.685872868500854e+21 | baseline: 0.07649341598316504
running experiment for lambda = 0.16666666666666666
	lambda 0.16666666666666666 | score 0.904 | grad 0.09508725639044689 | sklearn diff 0.05504062304161712
	IACV: 3745884260088878.5 | baseline: 0.0918429207517787
running experiment for lambda = 0.3333333333333333
	lambda 0.3333333333333333 | score 0.904 | grad 0.06411781909474411 | sklearn diff 0.05930458709538307
	IACV: 3.1507656880483866e+29 | baseline: 0.10637958837506639
running experiment for lambda = 0.5
	lambda 0.5 | score 0.9 | grad 0.08651504489459884 | sklearn diff 0.06050985802083103
	IACV: 6.981574554199988e+26 | baseline: 0.11587076443584113
running experiment for lambda = 0.6666666666666666
	lambda 0.6666666666666666 | score 0.908 | grad 0.04886014436897912 | sklearn diff 0.05985706014669606
	IACV: 1.3172311003404272e+22 | baseline: 0.124678

KeyboardInterrupt: 

In [ ]:
print(lbd_scores)
lbd_scores = np.asarray(lbd_scores)

In [ ]:
fig, ax = plt.subplots()
ax.plot(lambdas, lbd_scores[:, 0], label="IACV", c='black')
ax.plot(lambdas, lbd_scores[:, 1], label="baseline", c='green', linestyle="--")
ax.set_yscale('log')
ax.set_xlabel("$\lambda$")
ax.set_ylabel("Err Approx (log)")
ax.legend()
ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
#plt.savefig('svmtest_lambda_err_approx.pgf', bbox_inches='tight', pad_inches=0)